In [44]:
# imports

import requests
import os
import json
from pprint import pprint
from IPython.display import JSON
import pandas as pd

In [67]:
# Will find points of interest in Ladysmith BC, using the Wild Poppy Market latitude, longitude as the initial point of reference

wild_poppy = '48.99400256898083,-123.81788919678714'

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [ ]:
# Send the request through Postman first to try it out, and saved the response to a file.

In [71]:
# Send the request via Python as well.
# Refer to https://developer.foursquare.com/reference/place-search to get the authorization format, url, query parameters

# Set the keys
foursquare_api = os.environ["FOURSQUARE_API_KEY"]

# Get the url (copied from https://developer.foursquare.com/reference/place-search after entering the Wild Poppy latitude and longitude and the search radius of 1000m)
url = "https://api.foursquare.com/v3/places/search?ll=48.99400256898083%2C-123.81788919678714&radius=1000"

# Get the headers
headers = {
    "Accept": "application/json",
    "Authorization": foursquare_api
}

In [20]:
# Send the request
response = requests.get(url, headers=headers)

In [21]:
# Check the status of the response and print the response text
print(response.status_code)
print(response.text)

200
{"results":[{"fsq_id":"4b9f8746f964a520112937e3","categories":[{"id":13002,"name":"Bakery","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/bakery_","suffix":".png"}},{"id":13065,"name":"Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":77,"geocodes":{"main":{"latitude":48.993297,"longitude":-123.817749},"roof":{"latitude":48.993297,"longitude":-123.817749}},"link":"/v3/places/4b9f8746f964a520112937e3","location":{"address":"510 1st Ave","country":"CA","cross_street":"","formatted_address":"510 1st Ave, Ladysmith BC V9G 1R3","locality":"Ladysmith","neighborhood":["Cedar"],"postcode":"V9G 1R3","region":"BC"},"name":"Old Town Bakery","related_places":{},"timezone":"America/Vancouver"},{"fsq_id":"4d5dd55d5b276dcb151ef2c5","categories":[{"id":13034,"name":"Café","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/cafe_","suffix":".png"}},{"id":13049,"name":"Diner","icon":{"prefix":"https:

Parse through the response to get the POI details you want (rating, name, location, etc)

In [41]:
FS_response = response.json()

In [29]:
# Pretty print to view the response more easily

pprint(FS_response)

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 48.99400256898083,
                                                  'longitude': -123.81788919678714},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
                                       'suffix': '.png'},
                              'id': 13002,
                              'name': 'Bakery'},
                             {'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
                                       'suffix': '.png'},
                              'id': 13065,
                              'name': 'Restaurant'}],
              'chains': [],
              'distance': 77,
              'fsq_id': '4b9f8746f964a520112937e3',
              'geocodes': {'main': {'latitude': 48.993297,
                                    'longitude': -123.817749},
                           'ro

In [30]:
# Use JSON to get a different view of the response

JSON(FS_response)

<IPython.core.display.JSON object>

In [40]:
# Access the name of the first POI

FS_response['results'][0]['name']

'Old Town Bakery'

In [43]:
# identify all the points of interest returned by the Foursquare API within a 1000 m radius of the Wild Poppy

FS_POI = []

for item in FS_response['results']:
    FS_POI.append(item['name'])
    
print(FS_POI)

['Old Town Bakery', 'In the Bean Time', 'RBC Royal Bank', "Mr Popper's Sweet Shoppe", 'High Street Hair', 'Wild Poppy Bistro', 'Appetit', 'The Worldly Gourmet Kitchen Store', 'Salamander Books', 'Fox & Hound']


Put your parsed results into a DataFrame

In [46]:
# try to put the whole parsed response into a dataframe

pd.json_normalize(FS_response)

,results,context.geo_bounds.circle.center.latitude,context.geo_bounds.circle.center.longitude,context.geo_bounds.circle.radius
0,"[{'fsq_id': '4b9f8746f964a520112937e3', 'categ...",48.994003,-123.817889,1000


In [59]:
# not what I want, so access results and put into dataframe

results_df = pd.json_normalize(FS_response, record_path='results')
results_df.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.po_box,location.address_extended
0,4b9f8746f964a520112937e3,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",[],77,/v3/places/4b9f8746f964a520112937e3,Old Town Bakery,America/Vancouver,48.993297,-123.817749,48.993297,...,510 1st Ave,CA,,"510 1st Ave, Ladysmith BC V9G 1R3",Ladysmith,[Cedar],V9G 1R3,BC,NaN,NaN
1,4d5dd55d5b276dcb151ef2c5,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],42,/v3/places/4d5dd55d5b276dcb151ef2c5,In the Bean Time,America/Vancouver,48.994468,-123.817778,48.994468,...,18 High St,CA,,"18 High St, Ladysmith BC V9G 1B9",Ladysmith,[Cedar],V9G 1B9,BC,NaN,NaN
2,4caf56ad1463a143dbbc93a9,"[{'id': 11045, 'name': 'Bank', 'icon': {'prefi...",[{'id': 'd51a09f0-d890-0132-61d3-7a163eb2a6fc'...,13,/v3/places/4caf56ad1463a143dbbc93a9,RBC Royal Bank,America/Vancouver,48.993577,-123.817534,48.993577,...,527 1 St Ave,CA,,"527 1 St Ave, Ladysmith BC V9G 1A7",Ladysmith,NaN,V9G 1A7,BC,PO Box 1060,NaN
3,543065e3498ece744a874fb3,"[{'id': 17060, 'name': 'Candy Store', 'icon': ...",[],23,/v3/places/543065e3498ece744a874fb3,Mr Popper's Sweet Shoppe,America/Vancouver,48.994100,-123.818307,48.994100,...,32 High St,CA,1st Avenue,"32 High St (1st Avenue), Ladysmith BC V9G 1B9",Ladysmith,NaN,V9G 1B9,BC,NaN,# 1
4,4e3dac44fa76455375bbb10f,"[{'id': 11064, 'name': 'Hair Salon', 'icon': {...",[],27,/v3/places/4e3dac44fa76455375bbb10f,High Street Hair,America/Vancouver,48.994092,-123.817823,48.994092,...,23 High St,CA,,"23 High St, Ladysmith BC V9G 1B9",Ladysmith,NaN,V9G 1B9,BC,NaN,NaN


In [60]:
categories = results_df['categories']
categories

0    [{'id': 13002, 'name': 'Bakery', 'icon': {'pre...
1    [{'id': 13034, 'name': 'Café', 'icon': {'prefi...
2    [{'id': 11045, 'name': 'Bank', 'icon': {'prefi...
3    [{'id': 17060, 'name': 'Candy Store', 'icon': ...
4    [{'id': 11064, 'name': 'Hair Salon', 'icon': {...
5    [{'id': 13027, 'name': 'Bistro', 'icon': {'pre...
6    [{'id': 13003, 'name': 'Bar', 'icon': {'prefix...
7    [{'id': 17068, 'name': 'Gourmet Store', 'icon'...
8    [{'id': 17018, 'name': 'Bookstore', 'icon': {'...
9    [{'id': 13018, 'name': 'Pub', 'icon': {'prefix...
Name: categories, dtype: object

In [62]:
category_df = pd.json_normalize(categories)
category_df

,0,1,2
0,"{'id': 13002, 'name': 'Bakery', 'icon.prefix':...","{'id': 13065, 'name': 'Restaurant', 'icon.pref...",None
1,"{'id': 13034, 'name': 'Café', 'icon.prefix': '...","{'id': 13049, 'name': 'Diner', 'icon.prefix': ...",None
2,"{'id': 11045, 'name': 'Bank', 'icon.prefix': '...",None,None
3,"{'id': 17060, 'name': 'Candy Store', 'icon.pre...",None,None
4,"{'id': 11064, 'name': 'Hair Salon', 'icon.pref...",None,None
5,"{'id': 13027, 'name': 'Bistro', 'icon.prefix':...",None,None
6,"{'id': 13003, 'name': 'Bar', 'icon.prefix': 'h...","{'id': 13049, 'name': 'Diner', 'icon.prefix': ...","{'id': 13072, 'name': 'Asian Restaurant', 'ico..."
7,"{'id': 17068, 'name': 'Gourmet Store', 'icon.p...","{'id': 17069, 'name': 'Grocery Store / Superma...",None
8,"{'id': 17018, 'name': 'Bookstore', 'icon.prefi...",None,None
9,"{'id': 13018, 'name': 'Pub', 'icon.prefix': 'h...","{'id': 13065, 'name': 'Restaurant', 'icon.pref...",None


In [65]:
category_names = category_df['0']
category_names

KeyError: '0'

# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [68]:
# Send the request through Postman first to try it out, and saved the response to a file.

In [70]:
# Send the request via Python as well.
# Refer to https://www.yelp.com/developers/documentation/v3/authentication to get the API Key, authorization format
# Refer to https://www.yelp.com/developers/documentation/v3/business_search to get the url, query parameters

# Set the keys
yelp_api = 'Bearer '+os.environ["YELP_API_KEY"]

# Get the url (copied from url generated by Postman after entering the Wild Poppy latitude and longitude and the search radius of 1000m)
url = "https://api.yelp.com/v3/businesses/search?latitude=48.99400256898083&longitude=-123.81788919678714&radius=1000"

# Get the headers
headers = {
    "Accept": "application/json",
    "Authorization": yelp_api
}

KeyError: 'YELP_API_KEY'

Parse through your result and get POI details

Put your parsed results into a DataFrame

# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Get the top 10 restaurants according to their rating

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).